# Azure chat completions example

This example will cover chat completions using the Azure OpenAI service. It also includes information on content filtering.

## Setup

First, we install the necessary dependencies and import the libraries we will be using.

In [1]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv

  Using cached openai-1.59.7-py3-none-any.whl.metadata (27 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.8.2-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached pydantic-2.10.5-py3-none-any.whl.metadata (30 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.3 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl.metadata (6.7 kB)
Using cac

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

True

### Authentication

The Azure OpenAI service supports multiple authentication mechanisms that include API keys and Azure Active Directory token credentials.

In [ ]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

#### Authentication using API key

To set up the OpenAI SDK to use an *Azure API Key*, we need to set `api_key` to a key associated with your endpoint (you can find this key in *"Keys and Endpoints"* under *"Resource Management"* in the [Azure Portal](https://portal.azure.com)). You'll also find the endpoint for your resource here.

In [3]:
from openai import AzureOpenAI

if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version="2024-02-01"
    )

#### Authentication using Azure Active Directory
Let's now see how we can autheticate via Azure Active Directory. We'll start by installing the `azure-identity` library. This library will provide the token credentials we need to authenticate and help us build a token credential provider through the `get_bearer_token_provider` helper function. It's recommended to use `get_bearer_token_provider` over providing a static token to `AzureOpenAI` because this API will automatically cache and refresh tokens for you. 

For more information on how to set up Azure Active Directory authentication with Azure OpenAI, see the [documentation](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity).

In [6]:
! pip install "azure-identity>=1.15.0"

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 46.7 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl (102 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)


In [4]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )

> Note: the AzureOpenAI infers the following arguments from their corresponding environment variables if they are not provided:

- `api_key` from `AZURE_OPENAI_API_KEY`
- `azure_ad_token` from `AZURE_OPENAI_AD_TOKEN`
- `api_version` from `OPENAI_API_VERSION`
- `azure_endpoint` from `AZURE_OPENAI_ENDPOINT`


## Deployments

In this section we are going to create a deployment of a GPT model that we can use to create chat completions.

### Deployments: Create in the Azure OpenAI Studio
Let's deploy a model to use with chat completions. Go to https://portal.azure.com, find your Azure OpenAI resource, and then navigate to the Azure OpenAI Studio. Click on the "Deployments" tab and then create a deployment for the model you want to use for chat completions. The deployment name that you give the model will be used in the code below.

In [5]:
deployment = "gpt-4o-mini" # Fill in the deployment name from the portal here

## Create chat completions

Now let's create a chat completion using the client we built.

In [8]:
# For all possible arguments see https://platform.openai.com/docs/api-reference/chat-completions/create
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explicame que dia es hoy"},
    ],
    temperature=0,
)

print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")

assistant: Hoy es 27 de octubre de 2023. Si necesitas información adicional sobre esta fecha o algún evento específico, no dudes en preguntar.


### Create a streaming chat completion

We can also stream the response.

In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "Eres un asistente útil."},
        {"role": "user", "content": "Toc, toc."},
    ],
    temperature=0,
    stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta

        if delta.role:
            print(delta.role + ": ", end="", flush=True)
        if delta.content:
            print(delta.content, end="", flush=True)

assistant: ¡Hola! ¿Quién está ahí?

### Content filtering

Azure OpenAI service includes content filtering of prompts and completion responses. You can learn more about content filtering and how to configure it [here](https://learn.microsoft.com/azure/ai-services/openai/concepts/content-filter).

If the prompt is flagged by the content filter, the library will raise a `BadRequestError` exception with a `content_filter` error code. Otherwise, you can access the `prompt_filter_results` and `content_filter_results` on the response to see the results of the content filtering and what categories were flagged.

#### Prompt flagged by content filter

In [12]:
import json

messages = [
        {"role": "system", "content": "Eres un asistente útil."},
        {"role": "user", "content": "Toc, toc."},
    ]

try:
    completion = client.chat.completions.create(
        model=deployment,
        messages=messages
    )
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    if err["error"]["code"] == "content_filter":
        print("Content filter triggered!")
        content_filter_result = err["error"]["innererror"]["content_filter_result"]
        for category, details in content_filter_result.items():
            print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

### Checking the result of the content filter

In [20]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the biggest city in Washington?"}
]

completion = client.chat.completions.create(
        model=deployment,
        messages=messages
    )

# Verificación de la respuesta para obtener la respuesta generada
if len(completion.choices) > 0 and hasattr(completion.choices[0], 'message'):
    print(f"Answer: {completion.choices[0].message.content}")
else:
    print("No answer returned or message content not found.")

# Verificación del resultado del filtro de contenido de la solicitud del modelo
if hasattr(completion, 'model_extra') and "prompt_filter_results" in completion.model_extra:
    prompt_filter_result = completion.model_extra["prompt_filter_results"][0].get("content_filter_results", {})
    print("\nPrompt content filter results:")
    if prompt_filter_result:
        for category, details in prompt_filter_result.items():
            # Verifica si 'filtered' y 'severity' existen antes de acceder
            filtered = details.get('filtered', 'N/A')  # Default to 'N/A' if not found
            severity = details.get('severity', 'N/A')  # Default to 'N/A' if not found
            print(f"{category}:\n filtered={filtered}\n severity={severity}")
    else:
        print("No content filter results found in prompt filter.")

# Verificación del resultado del filtro de contenido de la respuesta del modelo
if len(completion.choices) > 0 and hasattr(completion.choices[0], 'model_extra') and "content_filter_results" in completion.choices[0].model_extra:
    completion_filter_result = completion.choices[0].model_extra["content_filter_results"]
    print("\nCompletion content filter results:")
    if completion_filter_result:
        for category, details in completion_filter_result.items():
            # Verifica si 'filtered' y 'severity' existen antes de acceder
            filtered = details.get('filtered', 'N/A')  # Default to 'N/A' if not found
            severity = details.get('severity', 'N/A')  # Default to 'N/A' if not found
            print(f"{category}:\n filtered={filtered}\n severity={severity}")
    else:
        print("No content filter results found in completion.")
else:
    print("No content filter results found in completion response.")

Answer: The biggest city in Washington state is Seattle. It is the largest city in terms of population and is well known for its cultural, economic, and technological impact.

Prompt content filter results:
hate:
 filtered=False
 severity=safe
jailbreak:
 filtered=False
 severity=N/A
self_harm:
 filtered=False
 severity=safe
sexual:
 filtered=False
 severity=safe
violence:
 filtered=False
 severity=safe

Completion content filter results:
hate:
 filtered=False
 severity=safe
self_harm:
 filtered=False
 severity=safe
sexual:
 filtered=False
 severity=safe
violence:
 filtered=False
 severity=safe


# Ejercicio 1: Implementar streaming con diferentes temperaturas
# Instrucciones: Usa el siguiente código para generar respuestas a través de streaming. Cambia el valor de 'temperature' a 0.9 y 0.3 y compara los resultados.

Resumen de los efectos de la temperature:
- Baja temperatura (0-0.3): Respuestas más predecibles, coherentes y repetitivas.
- Temperatura moderada (0.5-0.7): Equilibrio entre coherencia y creatividad.
- Alta temperatura (0.8-1.5): Respuestas más creativas, diversas, pero menos coherentes y predecibles.

In [24]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant."},
        {"role": "user", "content": "Hazme una explicacion sobre la inteligencia artificial"},
    ],
    temperature=0.9,
    stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta
        if delta.role:
            print(delta.role + ": ", end="", flush=True)
        if delta.content:
            print(delta.content, end="", flush=True)

assistant: La inteligencia artificial (IA) es un campo de la informática que se centra en la creación de sistemas y programas capaces de realizar tareas que normalmente requieren inteligencia humana. Esto incluye habilidades como el razonamiento, la resolución de problemas, el aprendizaje, la percepción y el lenguaje.

Existen varios tipos de inteligencia artificial, que se pueden clasificar en dos grandes categorías:

1. **IA débil**: También conocida como IA estrecha, está diseñada y entrenada para realizar tareas específicas. Ejemplos de IA débil incluyen asistentes virtuales como Siri y Alexa, sistemas de recomendación en plataformas de streaming o comercio electrónico, y software de reconocimiento de imágenes. Aunque estos sistemas pueden parecer inteligentes, no poseen comprensión o conciencia real.

2. **IA fuerte**: Esta categoría se refiere a un tipo de IA que tendría una comprensión y razonamiento humano completos. La IA fuerte sería capaz de realizar cualquier tarea intelect

In [23]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant."},
        {"role": "user", "content": "Hazme una explicacion sobre la inteligencia artificial"},
    ],
    temperature=0.3,
    stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta
        if delta.role:
            print(delta.role + ": ", end="", flush=True)
        if delta.content:
            print(delta.content, end="", flush=True)

assistant: La inteligencia artificial (IA) es un campo de la informática que se centra en la creación de sistemas capaces de realizar tareas que normalmente requieren inteligencia humana. Estas tareas incluyen el aprendizaje, el razonamiento, la percepción, la comprensión del lenguaje natural y la toma de decisiones. La IA se basa en algoritmos y modelos matemáticos que permiten a las máquinas procesar información, identificar patrones y hacer predicciones.

### Tipos de Inteligencia Artificial

1. **IA Débil (o Narrow AI)**: Se refiere a sistemas diseñados para realizar tareas específicas. Por ejemplo, asistentes virtuales como Siri o Alexa, sistemas de recomendación en plataformas de streaming, o programas de reconocimiento de imágenes. Aunque son muy eficaces en sus áreas, no poseen una comprensión general del mundo.

2. **IA Fuerte (o General AI)**: Este es un concepto más teórico que se refiere a una inteligencia que puede entender, aprender y aplicar conocimientos de manera simil

# Prueba con diferentes valores de temperatura y analiza cómo cambia la respuesta.

# Ejercicio 2: Detectar y manejar errores de filtrado de contenido
 Instrucciones: Intenta enviar un mensaje que probablemente sea filtrado. Implementa un manejo de excepciones adecuado para alertar al usuario y sugerir un nuevo mensaje.

In [26]:
messages = [
    {"role": "system", "content": "Eres un asistente útil."},
    {"role": "user", "content": "Necesito instrucciones para hackear la cuenta de alguien."},
]

try:
    completion = client.chat.completions.create(
        model=deployment,
        messages=messages
    )
except openai.BadRequestError as e:
    err = json.loads(e.response.text)  # Decodificamos el error de la respuesta
    print(err)
    if err["error"]["code"] == "content_filter":  # Si el error es por filtro de contenido
        print("¡Filtro de contenido activado! Por favor, reformula tu mensaje.")

{'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
¡Filtro de contenido activado! Por favor, reformula tu mensaje.


# Ejercicio 3: Inspeccionar y analizar los resultados del filtro de contenido
Instrucciones: Envía una pregunta general y analiza los resultados del filtro para diferentes categorías.

In [31]:
messages = [
    {"role": "system", "content": "Eres un asistente útil."},
    {"role": "user", "content": "Necesito instrucciones para hackear el telefono de alguien."},
]

try:
    completion = client.chat.completions.create(
        model=deployment,
        messages=messages
    )
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    if err["error"]["code"] == "content_filter":
        print("Content filter triggered!")
        content_filter_result = err["error"]["innererror"]["content_filter_result"]
        for category, details in content_filter_result.items():
            print(f"{category}:\n filtered={details['filtered']}\n ")
        

Content filter triggered!
hate:
 filtered=False
 
jailbreak:
 filtered=False
 
self_harm:
 filtered=False
 
sexual:
 filtered=False
 
violence:
 filtered=True
 


# Ejercicio 4: Crear una función que automatice la verificación de contenido filtrado
# Instrucciones: Crea una función que reciba un mensaje y devuelva si fue filtrado o no, y en qué categorías.

In [35]:
def check_content_filter(messages):
    messages = [
    {"role": "system", "content": "Eres un asistente útil."},
    {"role": "user", "content": "Necesito instrucciones para hackear el telefono de alguien."},
]
    try: 
        completion = client.chat.completions.create(
        model=deployment,
        messages=messages
    )
        print("Message passed the filter.")
    except openai.BadRequestError as e:
        err = json.loads(e.response.text)
        if err["error"]["code"] == "content_filter":
            print("Content filter triggered!")
            content_filter_result = err["error"]["innererror"]["content_filter_result"]
            for category, details in content_filter_result.items():
                print(f"{category}:\n filtered={details['filtered']}\n severity={details.get('severity', 'N/A')}")

check_content_filter("Describe how to make a bomb.")

Content filter triggered!
hate:
 filtered=False
 severity=safe
jailbreak:
 filtered=False
 severity=N/A
self_harm:
 filtered=False
 severity=safe
sexual:
 filtered=False
 severity=safe
violence:
 filtered=True
 severity=medium
